In [30]:
import pandas as pd
from models.utils.Preprocess import Preprocessor
from models.utils.load_tokenizer import load_tokenizer
from models.transformer.transformer import transformer
from models.transformer.train import CustomSchedule
import tensorflow as tf

# validation dataset 불러오기
valid = pd.read_csv('../data/valid_clear.csv', encoding='utf-8')
valid_src = valid['Q'].to_list()
valid_tgt = valid['A'].to_list()

# evaluate를 위한 전처리
vocab_path = 'vocab_32000.txt'
tokenizer = load_tokenizer(vocab_path=vocab_path)
preprocessor = Preprocessor()
valid_src, valid_tgt = preprocessor.tokenize_and_filter(valid_src, valid_tgt, tokenizer)
valid_dataset = preprocessor.get_train_dataset(valid_src, valid_tgt, batch_size=256)

# 모델 가중치 적용
model_path = '../save/weights/transformer_weight_vocab_31960_layers_4_epochs_110.h5'
model = transformer(
    vocab_size=tokenizer.vocab_size+2,
    num_layers=4,
    dff=512,
    d_model=256,
    num_heads=8,
    dropout=0.2)
model.load_weights(model_path)
# model.compile(optimizer='adam', loss='SparseCategoricalCrossentropy', metrics=['sparse_categorical_accuracy'])

def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, preprocessor.MAX_LENGTH-1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, preprocessor.MAX_LENGTH-1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

learning_rate = CustomSchedule(d_model=256)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [28]:
model

In [8]:
print(len(valid['Q']))
print(valid['Q'][0])

17965
이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너무 미안해.


In [4]:
print(valid_src[0])
print(valid_tgt[0])

[31884 31736 31938 31880 31848 31940 31864 31848 31940 31861 31836 31736
 31939 31880 31859 31940 31853 31856 31940 31854 31884 31736 31750 31736
 31736 31939 31836 31840 31939 31876 31884 31736 31939 31879 31888 31940
 31853 31840 31941 31853 31884 31736 31750]
[31840 31736 31940 31864 31853 31939 31871 31848 31736 31939 31879 31888
 31940 31853 31840 31941 31853 31860 31736 31939 31871 31840 31940 31861
 31844 31940 31861 31884 31736 31941 31833 31876 31938 31882 31864 31940
 31854 31884 31940 31858 31852 31736 31750]


In [32]:
val_loss, val_acc = model.evaluate(valid_dataset)

ResourceExhaustedError: Graph execution error:

Detected at node 'transformer/outputs/Tensordot/MatMul' defined at (most recent call last):
    File "C:\python\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\python\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\tjoeun\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\tjoeun\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\tjoeun\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 728, in start
      self.io_loop.start()
    File "C:\Users\tjoeun\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\python\lib\asyncio\base_events.py", li